In [15]:
#!/bin/bash/python3.7
# -*- coding:utf-8 -*-

from PIL import Image
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import urllib.request
from selenium.webdriver.common.keys import Keys
import time
import re

#############################브라우저를 테스트하는 인스턴스 생성############
drv = webdriver.Chrome('C:\\Users\\you36\\Desktop\\capstone\\chromedriver') #현재 크롬 버전과 맞는 크롬 드라이버 위치
# ex : 'C:\\Users\\qhxma\\chromedriver'

keyword_list = ["횡단보도앞"]


####### 키워드와 구글 이미지 검색 url을 결합하고 해당 검색 창을 띄움###
for keyword in keyword_list:
    
    search_engine_url = "https://www.bing.com/images/search?q="+keyword
    print(search_engine_url)
    drv.get(search_engine_url)

    #search_box = drv.find_element_by_name('q') #검색창 선택
    #search_box.send_keys(keyword)# 키워드 입력
    #search_box.submit()#엔터 -> 이미지 검색화면으로 옮기기엔 위의 방법이 더 빨라서 잠시 주석걸어둠

    #search_page = drv.page_source #

    #soup=bs(second_page, 'html.parser') # 검색창에서 이미지 검색창으로 옮기기 위한 url을 받아오기 위한거지만 주석걸어둠

    ###############################  스크롤 끝까지 내려서 문서 완성시키기 ###########

    for i in range(1,10): # 1,30은 30번 내릴려고 하는거. 이미지 많이 받으려면 바꿔야함.
        drv.find_element_by_xpath("//body").send_keys(Keys.END) #스크롤을 밑바닥까지
        time.sleep(0.5)

    #######################################################################    
    #menu_list = soup.select("#hdtb-msb-vis > div:nth-child(2) > a")


    #이미지 소스 정보 리스트 작성 
    params = [] #이미지 원본 주소 넣을 빈 리스트
    search_page_html = drv.page_source
    pattern = "[>|\s]\d{2,7}"#>나 공백이 숫자(1자리~5자리) 바로 앞에 있는 패턴만 가져오기
    #옆의 것에서 숫자 2개만 뽑아오게됨 <a id="msz" target="_blank" href="/images/search?q=%25ec%2597%25ac%25ec%259e%2590&amp;cbir=ms&amp;rxc=12&amp;sbirxc=30&amp;mid=DBF16767D97D177C71611A37854C459EB4FA96FE&amp;simid=0&amp;vw=e6be4%20e6bdb%20c06d1%20860de%2014729%2087d49%206756a%20c0994%20181b8%206d161%207339d%20c6be4%2037497%208fc2f%2044de8%20c6c1f%20dfc14%20cbd79%2080e83%2084952%20850a45f8eb1e8a3d70d5dc71255730af70ed580dca955061668550b3522cd711d354e98d399dad875abcb32e52f0114dd384&amp;FORM=IMSFRD" data-tooltip="크기 더 보기">1000 × 1500 jpeg</a>
    soup = bs(search_page_html, "html.parser") #현제 페이지 파서 가져오기(검색창)
    img_links = soup.find_all("a",class_='iusc')# 이미지 링크 다 가져오기
    count = 0;
    for list in img_links:
        drv.execute_script('window.open("about:blank","_blank");')#빈 탭 열기
        tabs = drv.window_handles#탭 핸들러가져오기
        drv.switch_to.window(tabs[1])#두번째 탭으로 핸들러 옮김
        drv.get("https://bing.com"+list.get('href'))#두번 째 탭에서 앞의 이미지 링크 열기
        
        time.sleep(0.5) # 페이지 자원이 충분히 완성될 때까지 대기
        new_tab = drv.page_source #이미지 링크로 열린 페이지 가져오기
        new_soup = bs(new_tab,"html.parser") #새로운 탭의 파서 가져오기
        img_src = new_soup.find("img",class_="nofocus")#원본 이미지 url 가져오기

        size_info = new_soup.find('a',id="msz")#이미지 크기 정보 가져오기
        size_list = re.findall(pattern, str(size_info)) #이미지 사이즈 정보만 추출(문자열 좀더 가다듬어야됨)
        
        size_list[0] = size_list[0].replace('>','')#아직 정리 안된 >문자 제거
        size_list[1] = size_list[1].strip()#아직 제거 안된 공백문자 제거
        width = int(size_list[0]) #너비 정보 정수로 변환
        height = int(size_list[1])#높이 정보 정수로 변환
        #print("width: " + str(width), "height: " + str(height))
        if width>=50 and height>=50: # 크기가 512 x 512 이상인 경우만 
            try:
                params.append(img_src.get('src'))
            except KeyError:
                params.append(img_src.get('data-src'))
        drv.close()
        drv.switch_to.window(tabs[0])

    #r을 문자열 앞에 붙이면 raw로 공백이든 뭐든 다 문자열로 받아주나 봄
    store_loc=r"C:\\Users\\you36\\Desktop\\capstone\\bingImage\\"
    #r"C:\\Users\\qhxma\\OneDrive\\바탕 화면\\밑바닥부터딥러닝\\crawl\\test\\"
    file_name=keyword
    #이미지 저장하기
    for id, loc in enumerate(params,1): # id 는 1부터, loc에는 차례대로 이미지 주소가 들어감
        try:
            if loc.endswith('PNG') or loc.endswith('png'):
                name, header = urllib.request.urlretrieve(loc,store_loc + file_name + str(id)+".png")
            elif loc.endswith('JPG') or loc.endswith('jpg') or loc.endswith('jpeg') or loc.endswith('JPEG'):
                name, header = urllib.request.urlretrieve(loc,store_loc + file_name + str(id)+".jpg")    
            else :
                name, header = urllib.request.urlretrieve(loc,store_loc + file_name + str(id)+".jpg")
            time.sleep(0.5)
        except Exception as e:#에러 처리 http 403 forbidden 같은 예외 처리
            print("------------------------------------")
            print(f"{id+1}번째 이미지 Download failed!")
            print(e)
        else:
            # 성공 시 response 상태정보 확인
            print("----------------------------------------------------")
            print(f"{id+1}번째 file dowload path: {name}")
            print("----------------------------------------------------")
            print(f"{id+1}번째 Header Info :")
            print(header)
            

drv.close()

https://www.bing.com/images/search?q=횡단보도앞
----------------------------------------------------
2번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞1.jpg
----------------------------------------------------
2번째 Header Info :
Date: Thu, 28 May 2020 12:00:51 GMT
Server: Apache/2.2.15 (CentOS)
Last-Modified: Mon, 12 Feb 2018 03:48:40 GMT
ETag: "1a00449-120115-564fbc0f1cbe5"
Accept-Ranges: bytes
Content-Length: 1179925
Connection: close
Content-Type: image/jpeg


----------------------------------------------------
3번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞2.jpg
----------------------------------------------------
3번째 Header Info :
Date: Thu, 28 May 2020 12:00:52 GMT
Server: Apache/2.2.15 (CentOS)
Last-Modified: Mon, 12 Feb 2018 03:48:42 GMT
ETag: "1a0049b-13032e-564fbc11190d8"
Accept-Ranges: bytes
Content-Length: 1245998
Connection: close
Content-Type: image/jpeg


----------------------------------------------------
4번째 file dowlo

----------------------------------------------------
17번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞16.jpg
----------------------------------------------------
17번째 Header Info :
Server: nginx
Date: Thu, 28 May 2020 12:01:01 GMT
Content-Type: image/jpeg
Content-Length: 444235
Last-Modified: Thu, 27 Dec 2018 08:06:58 GMT
Connection: close
ETag: "5c248822-6c74b"
Accept-Ranges: bytes


----------------------------------------------------
18번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞17.jpg
----------------------------------------------------
18번째 Header Info :
Date: Thu, 28 May 2020 12:01:05 GMT
Content-Type: image/jpeg
Last-Modified: Tue, 18 Jun 2019 08:12:18 GMT
Accept-Ranges: bytes
ETag: "22b3ef8bad25d51:0"
Server: Microsoft-IIS/10.0
X-Powered-By: OJS19
Content-Length: 92368
X-Cache: HIT from cache.xcdn.uplus.co.kr
X-Cache: MISS from i1.cache.xcdn.uplus.co.kr
Connection: close


-----------------------------------------------

----------------------------------------------------
32번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞31.jpg
----------------------------------------------------
32번째 Header Info :
Server: openresty
Date: Thu, 28 May 2020 12:01:18 GMT
Content-Type: image/jpeg
Content-Length: 94498
Connection: close
Expires: Thu, 28 May 2020 14:01:18 GMT
Last-Modified: Sat, 18 Jan 2020 04:22:14 GMT
Accept-Ranges: bytes
X-WCSS: dC1pbWcwMS1id2NhY2hlMTk6MDpjaHR0cDoxNg==
Cache-Control: max-age=7200


----------------------------------------------------
33번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞32.jpg
----------------------------------------------------
33번째 Header Info :
Date: Thu, 28 May 2020 12:01:19 GMT
Server: PWS/8.3.2.7
X-Px: ms h0-s5.p77-icn ( h0-s10.p77-icn), ht h0-s10.p77-icn.cdngp.net
Age: 1429
Cache-Control: max-age=21600
Expires: Thu, 28 May 2020 17:37:30 GMT
Accept-Ranges: bytes
Content-Length: 175820
Content-Type: image/jpeg
Last-M

----------------------------------------------------
47번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞46.jpg
----------------------------------------------------
47번째 Header Info :
Accept-Ranges: bytes
access-control-allow-origin: *
access-control-expose-headers: Content-Length
Age: 229627
cache-control: max-age=604800, must-revalidate
Content-Type: image/jpeg
Date: Thu, 28 May 2020 12:01:34 GMT
Last-Modified: Fri, 28 Apr 2017 01:53:44 GMT
Server: ECS (hhp/9AD2)
strict-transport-security: max-age=631138519
surrogate-key: media media/bucket/0 media/857775306228211712
X-Cache: MISS
x-connection-hash: 42ba0b7d6d106bbcaa5d51e25a57e9b6
x-content-type-options: nosniff
x-response-time: 373
Content-Length: 71570
Connection: close


----------------------------------------------------
48번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞47.jpg
----------------------------------------------------
48번째 Header Info :
Content-Type: image/jpeg
Acce

----------------------------------------------------
61번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞60.jpg
----------------------------------------------------
61번째 Header Info :
Server: nginx
Date: Thu, 28 May 2020 12:02:04 GMT
Content-Type: image/jpeg
Content-Length: 111062
Last-Modified: Tue, 30 Oct 2012 06:51:10 GMT
Connection: close
ETag: "508f78de-1b1d6"
Accept-Ranges: bytes


----------------------------------------------------
62번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞61.jpg
----------------------------------------------------
62번째 Header Info :
Date: Thu, 28 May 2020 12:02:07 GMT
Content-Length: 47335
Accept-Ranges: bytes
ETag: "5b7caa25:25d83"
Last-Modified: Wed, 22 Aug 2018 00:11:17 GMT
Content-Type: image/jpeg
Server: Apache/2.4.25 (Unix)
Connection: Keep-Alive
Keep-Alive: timeout=1
Age: 98517
Connection: close


----------------------------------------------------
63번째 file dowload path: C:\\Users\\you36\\Des

...

----------------------------------------------------
182번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞181.jpg
----------------------------------------------------
182번째 Header Info :
Content-Length: 163647
Expires: Sat, 30 May 2020 12:05:02 GMT
Date: Thu, 28 May 2020 12:05:02 GMT
Last-Modified: Thu, 25 Jun 2015 09:00:44 GMT
Content-Type: image/jpeg
Cache-Control: max-age=172800
Accept-Ranges: bytes
Content-Disposition: inline; filename="IMG_3702-1.jpg"; filename*=UTF-8''IMG_3702-1.jpg
X-WCSS: dC1jb21tb24wMS1id2NhY2hlNDowOmNodHRwOjI3
Via: 1.1 Wcache(3.1), 1.1 Wcache(3.1)
Connection: close


----------------------------------------------------
183번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞182.jpg
----------------------------------------------------
183번째 Header Info :
Content-Type: image/jpeg
Access-Control-Allow-Origin: *
Access-Control-Allow-Headers: *
Timing-Allow-Origin: *
Report-To: {"group":"network-errors","max_age":6048

----------------------------------------------------
194번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞193.jpg
----------------------------------------------------
194번째 Header Info :
Accept-Ranges: bytes
access-control-allow-origin: *
access-control-expose-headers: Content-Length
Age: 615
cache-control: max-age=604800, must-revalidate
Content-Type: image/jpeg
Date: Thu, 28 May 2020 12:05:21 GMT
Last-Modified: Fri, 29 Dec 2017 01:10:56 GMT
Server: ECS (sgb/C68B)
strict-transport-security: max-age=631138519
surrogate-key: media media/bucket/1 media/946549562151354369
X-Cache: HIT
x-connection-hash: d887ac8c88c2dc933d6eb4375c1fba85
x-content-type-options: nosniff
x-response-time: 269
Content-Length: 79200
Connection: close


----------------------------------------------------
195번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞194.jpg
----------------------------------------------------
195번째 Header Info :
Content-Type: image/jpeg
La

----------------------------------------------------
208번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞207.jpg
----------------------------------------------------
208번째 Header Info :
Content-Type: image/jpeg
Access-Control-Allow-Origin: *
Access-Control-Allow-Headers: *
Timing-Allow-Origin: *
Report-To: {"group":"network-errors","max_age":604800,"endpoints":[{"url":"https://aefd.nelreports.net/api/report?cat=bingth"}]}
NEL: {"report_to":"network-errors","max_age":604800,"success_fraction":0.001,"failure_fraction":1.0}
Content-Length: 16227
Cache-Control: public, max-age=1209600
Date: Thu, 28 May 2020 12:05:44 GMT
X-Cache: TCP_MISS from a104-86-182-71.deploy.akamaitechnologies.com (AkamaiGHost/10.0.2.3-29612037) (-)
Connection: close
X-Forward-Proto: http
CDN-Origin-Protocol: HTTP
X-Check-Cacheable: YES


----------------------------------------------------
209번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞208.jpg
----------------

----------------------------------------------------
223번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞222.jpg
----------------------------------------------------
223번째 Header Info :
Date: Thu, 28 May 2020 12:05:54 GMT
Server: PWS/8.3.2.7
X-Px: ms h0-s16.p71-icn ( h0-s4.p71-icn), rf-ht h0-s4.p71-icn ( h0-s783.p61-icn>CONN), ht-d h0-s783.p61-icn.cdngp.net
Age: 100282
Cache-Control: max-age=172800
Expires: Fri, 29 May 2020 08:14:32 GMT
Accept-Ranges: bytes
Content-Length: 107060
Content-Type: image/jpeg
Last-Modified: Thu, 23 Apr 2015 09:52:15 GMT
Connection: close


----------------------------------------------------
224번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞223.jpg
----------------------------------------------------
224번째 Header Info :
Content-Type: image/jpeg
Content-Length: 440644
Connection: close
Accept-Ranges: bytes
Date: Thu, 28 May 2020 12:05:59 GMT
Last-Modified: Thu, 16 Jun 2016 23:36:04 GMT
Server: Microsoft-I

----------------------------------------------------
236번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞235.jpg
----------------------------------------------------
236번째 Header Info :
Date: Thu, 28 May 2020 12:06:13 GMT
Content-Type: image/jpeg
Last-Modified: Mon, 24 Dec 2012 10:01:41 GMT
Accept-Ranges: bytes
ETag: "3ed2c0abbde1cd1:0"
Server: Microsoft-IIS/10.0
X-Powered-By: OJS11
Content-Length: 104047
X-Cache: HIT from cache.xcdn.uplus.co.kr
X-Cache: MISS from i3.cache.xcdn.uplus.co.kr
Connection: close


----------------------------------------------------
237번째 file dowload path: C:\\Users\\you36\\Desktop\\capstone\\bingImage\\횡단보도앞236.jpg
----------------------------------------------------
237번째 Header Info :
ETag: "3bfacc110d9cf897880cf70c004a8f4b"
Accept-Ranges: bytes
Content-Type: image/jpeg
Content-Length: 188095
Connection: close
Vary: Origin
Cache-Control: immutable, max-age=31536000
X-CDN: akamai


-----------------------------------------------